In [1]:
!pip install spacy_transformers
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 57.4 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.4
    Uninstalling spacy-3.5.4:
      Successfully uninstalled spacy-3.5.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.6.0 which is incompatible.


In [2]:
import spacy
from spacy.tokens import  DocBin
from tqdm import tqdm
import json

In [3]:
spacy.__version__

'3.6.0'

In [4]:
!nvidia-smi

Sat Jul 15 12:20:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
! git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 82 (delta 14), reused 74 (delta 14), pack-reused 6
Unpacking objects: 100% (82/82), 5.62 MiB | 7.41 MiB/s, done.


In [6]:
cv_data= json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json','r'))

In [7]:
len(cv_data)

200

In [8]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

2023-07-15 12:20:46.698651: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
cv_data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [10]:
def get_spacy_doc(file,data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents=[]
    entity_indices = []
    for start,end, label in annot:
      skip_entity=False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
        continue
      entity_indices = entity_indices + list(range(start,end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode = 'strict')
      except:
        continue
      if span is None:
        err_data=str([start,end]) + '  ' +str(text) +"\n"
        file.write(err_data)

      else:
        ents.append(span)

    try:
      doc.ents= ents
      db.add(doc)
    except:
      pass
  return db


In [11]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data,test_size=0.3)

In [12]:
len(train), len(test)

(140, 60)

In [13]:
file = open('error.txt', 'w')
db = get_spacy_doc(file,train)
db.to_disk('traindata.spacy')
db=get_spacy_doc(file,test)
db.to_disk('testdata.spacy')
file.close()

100%|██████████| 60/60 [00:00<00:00, 91.30it/s]


In [ ]:
len(db.tokens)

60

In [16]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train ./traindata.spacy --paths.dev ./testdata.spacy --gpu-id 0

2023-07-15 12:49:14.959751: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-07-15 12:49:22,895] [INFO] Set up nlp object from config
[2023-07-15 12:49:22,911] [INFO] Pipeline: ['transformer', 'ner']
[2023-07-15 12:49:22,914] [INFO] Created vocabulary
[2023-07-15 12:49:22,914] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Ro

In [23]:
nlp=spacy.load('/content/output/model-best')
doc = nlp('my name is Kishor Chandra Sahu. I worked at TechMahindra. I have 12 years of experience')
for ent in doc.ents:
  print(ent.text,"   ---->>>>>", ent.label_)

Kishor Chandra Sahu    ---->>>>> Name


In [18]:
!pip install PyMuPDF


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 90.3 MB/s eta 0:00:00


In [21]:
import sys,fitz

In [24]:
filename = '/content/CV-Parsing-using-Spacy-3/data/test/Alice Clark CV.pdf'
doc = fitz.open(filename)


In [26]:
text=""
for page in doc:
	text = text + str(page.get_text())

text = text.strip()
text = ' '.join(text.split('\n'))
text

'Alice Clark  AI / Machine Learning    Delhi, India Email me on Indeed  •  20+ years of experience in data handling, design, and development  •  Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to  data warehousing and business intelligence  •  Database: Experience in database designing, scalability, back-up and recovery, writing and  optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes.  Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure,  Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake  analytics(U-SQL)  Willing to relocate anywhere    WORK EXPERIENCE  Software Engineer  Microsoft – Bangalore, Karnataka  January 2000 to Present  1. Microsoft Rewards Live dashboards:  Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping  online. Microsoft Rewards members can earn points when searching with Bing, br

In [27]:
doc = nlp(text)
for ent in doc.ents:
  print(ent.text,"   ---->>>>>", ent.label_)

AI / Machine Learning    ---->>>>> Degree
Delhi    ---->>>>> Location
Microsoft    ---->>>>> Companies worked at
Software Engineer    ---->>>>> Designation
Microsoft    ---->>>>> Companies worked at
Microsoft    ---->>>>> Companies worked at
Microsoft    ---->>>>> Companies worked at
Microsoft    ---->>>>> Companies worked at
Microsoft    ---->>>>> Companies worked at
Microsoft    ---->>>>> Companies worked at
Microsoft    ---->>>>> Companies worked at
Indian Institute of Technology    ---->>>>> College Name
Machine Learning, Natural Language Processing, and Big Data Handling    ---->>>>> Skills
